In [5]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
plt.style.use('seaborn-whitegrid')

# Gradient Descent for a Linear Model

In [21]:
def linear_model(X, W):
    # W is column vector
    return X @ W[1:, :] + W[0, :]

def rmse(model, X, T, W):
    Y = model(X, W)
    return np.sqrt(np.mean((T - Y)**2))



Let the output of our model be $Y$ and the error being minimized $E$.

To perform gradient descent, we need $\frac{\partial E}{\partial W}$. The calculation of this can be expressed with the chain rule.

$$
\frac{\partial E}{\partial W} = \frac{\partial E}{\partial Y} \frac{\partial Y}{\partial W}
$$

The error we want to minimize is the squared error, $(T - Y)^2$ and $Y = XW$.

$$
\begin{align}
\frac{\partial E}{\partial W} &= \frac{\partial E}{\partial Y} \frac{\partial Y}{\partial W} \\
\frac{\partial E}{\partial W} &= \frac{\partial (T - Y)^2}{\partial Y} \frac{\partial X W}{\partial W} \\
\frac{\partial E}{\partial W} &= -2 (T - Y) X 
\end{align}
$$

Given a simple model.

$$
\begin{align*}
   g(x;w) &= w_0 + w_1 x_1 + w_2 x_2 + \cdots + w_D x_D \\
   &= w_0 + \sum_{i=1}^D w_i x_i \\
   & = \sum_{i=0}^D w_i x_i \text{, where } x_0 = 1 \\
   &= w^T x\\
   &= x^T w
\end{align*}
$$

Force exerted by a spring is proportional to its length. The potential energy stored in the spring is proportional to the square of its length. So the position that minimizes the potential energy in the springs:

\begin{align*}
\sum_{n=1}^N (t_n - g(x_n;w))^2
\end{align*}

So for a model g:

$$
g(x; w) = w_0 + w_1 x
$$

with parameters $$w = (w_0, w_1)$$ what gives the best fit?

$$
\hat{w} = \text{argmin}{w} \sum_{n=1}^N (t_n - g(x_n ; w))^2
$$

Collect all targets into matrix $T$ and $x$ samples into matrix $X$ where $N$ is the number of samples and $D is the sample dimensionality.


$$
\begin{align*}
    T &= \begin{bmatrix}
      t_1 \\ t_2 \\ \vdots \\ t_N
    \end{bmatrix} \\
    X &= \begin{bmatrix}
      x_{1,0} & x_{1,1} & x_{1,2} & \dotsc & x_{1,D} \\
      x_{2,0} & x_{2,1} & x_{2,2} & \dotsc & x_{2,D} \\
      \vdots \\
      x_{N,0} & x_{N,1} & x_{N,2} & \dotsc & x_{N,D}
    \end{bmatrix}\\
    w &= \begin{bmatrix} w_0 \\ w_1 \\ \vdots \\ w_D \end{bmatrix}
  \end{align*}
  $$

Collection of all differences is $T−Xw$, which is an $N×1$ matrix. To form the square of all values and add them up, just do a dot product $(T−Xw)^T(T−Xw)$. 

This only works if the value we are predicting is a scalar, which means $T$ is a column matrix. If we want to predict more than one value for each sample, $T$ will have more than one column. Let's continue with the derivation assuming $T$ has $K$ columns, meaning we want a linear model with $K$ outputs.

To find the best value for $w$, we take the derivative of the sum of squared error objective, set it equal to 0 and solve for $w$. 

Here $xn$ is one sample as a column vector, so it must be transposed to a row vector before being multiplied by the column vector $w$.

\begin{align*}
\frac{\partial \sum_{n=1}^N (t_n - g(x_n;w))^2}{\partial w} &= -2 \sum_{n=1}^N (t_n - g(x_n;w) \frac{\partial g(x_n;w)}{\partial w}\\
&= -2 \sum_{n=1}^N (t_n - x_n^T w) \frac{\partial x_n^T w}{\partial w}\\
&= -2 \sum_{n=1}^N (t_n - x_n^T w) x_n^T
\end{align*}

We can then express the sum with a dot product:

$$
\begin{align*}
\frac{\partial \sum_{n=1}^N (t_n - g(x_n;w))^2}{\partial w} 
&= -2 \sum_{n=1}^N (t_n - x_n^T w) x_n^T\\
&= -2 X^T (T - X w)
\end{align*}
$$


Now we set this all to 0 and solve for w.

\begin{align*}
-2 X^T (T - X w) &= 0\\
X^T (T - X w) &= 0\\
X^T T &= X^T X w\\
w &= (X^T X)^{-1} X^T T
\end{align*}


## Incremental algorithm

But what if we have millions of samples? $X$ and $T$ can get very large.

We can derive a sequential algorithm for finding $w$, as the derivative of a sum is the sum of the derivatives. We can express this as a gradient which is a vector or matrix of derivatives.

\begin{align*}
g(x_n, w) &= w_0 + w_1 x_{n,1} + w_2 x_{n,2} + \cdots + w_D x_{n,D} = x_n^T w\\
E(X, T, w) &= \sum_{n=1}^N (t_n - g(x_n, w))^2\\
\nabla_w E(X, T, w) &= \nabla_w \left ( \sum_{n=1}^N (t_n - g(x_n, w))^2 \right )\\
&= 
\sum_{n=1}^N \nabla_w (t_n - g(x_n, w))^2\\
&= 
\sum_{n=1}^N 2 (t_n - g(x_n, w)) \nabla_w (t_n - g(x_n, w)) \\
&= 
\sum_{n=1}^N 2 (t_n - g(x_n, w)) (-1) \nabla_w g(x_n, w) \\
&= 
\sum_{n=1}^N 2 (t_n - g(x_n, w)) (-1) \nabla_w (x_n^T w) \\
&= 
\sum_{n=1}^N 2 (t_n - g(x_n, w)) (-1) x_n \\
&= 
-2 \sum_{n=1}^N (t_n - g(x_n, w))  x_n \\
\end{align*}

Now the weights for the next time step are updated based on the gradient of $E$ for that sample. This is called stochastic approximation where $\rho$ is a scalar step size.

\begin{align*}
w^{(k+1)} &= w^{(k)}  + \rho \; x_n \; (t_n^T - g(x_n, w))) 
\end{align*}